In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
from ase.db import connect
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Molecule

def add_selective_dynamics_to_slab(slab, fixed_fraction=0.1):
    z_coords = [site.coords[2] for site in slab]
    z_min = min(z_coords)
    z_max = max(z_coords)
    threshold = z_min + fixed_fraction * (z_max - z_min)
    
    sd_flags = []
    for site in slab:
        if site.coords[2] <= threshold:
            sd_flags.append([False, False, False])
        else:
            sd_flags.append([True, True, True])
    slab.add_site_property("selective_dynamics", sd_flags)
    return slab

def add_adsorbate_model(slab, adsorbate, height=2.0):
    slab_z_max = max([site.coords[2] for site in slab])
    x_coords = [site.coords[0] for site in slab]
    y_coords = [site.coords[1] for site in slab]
    center_x = (min(x_coords) + max(x_coords)) / 2.0
    center_y = (min(y_coords) + max(y_coords)) / 2.0

    adsorbate_com = adsorbate.center_of_mass
    dx = center_x - adsorbate_com[0]
    dy = center_y - adsorbate_com[1]
    dz = (slab_z_max + height) - adsorbate_com[2]
    translation_vector = [dx, dy, dz]
    
    adsorbate_translated = adsorbate.copy()
    adsorbate_translated.translate_sites(range(len(adsorbate_translated)), translation_vector)
    
    combined_structure = slab.copy()
    for site in adsorbate_translated.sites:
        combined_structure.append(
            site.specie,
            site.coords,
            coords_are_cartesian=True,
            properties={"selective_dynamics": [True, True, True]}
        )
    return combined_structure


def main():
    atomic_species = ["S", "S", "F", "F", "F", "F", "F", "F", "O", "O", "O", "O", "N", "C", "C", "Li"]
    atomic_coords = [
        [-0.7145,  0.2062,  0.0000],
        [ 0.7145,  0.2062,  0.0000],
        [-1.4720, -0.9455,  0.0000],
        [-2.2970, -0.1205,  0.0000],
        [-1.4720,  0.7045,  0.0000],
        [ 1.4720,  0.7045,  0.0000],
        [ 2.2970, -0.1205,  0.0000],
        [ 1.4720, -0.9455,  0.0000],
        [-0.7145,  1.0313,  0.0000],
        [-0.7145, -0.6187,  0.0000],
        [ 0.7145,  1.0313,  0.0000],
        [ 0.7145, -0.6187,  0.0000],
        [ 0.0000, -0.2062,  0.0000],
        [-1.4720, -0.1205,  0.0000],
        [ 1.4720, -0.1205,  0.0000],
        [ 0.0000, -1.0313,  0.0000],
    ]
    adsorbate = Molecule(atomic_species, atomic_coords)
    
    input_db_file = "initial_db.db"
    in_db = connect(input_db_file)
    
    output_db_file = "adsorbed.db"
    out_db = connect(output_db_file)
    
    rows = list(in_db.select())
    
    for row in rows:
        try:
            atoms = row.toatoms()
            structure = AseAtomsAdaptor.get_structure(atoms)
        except Exception as e:
            print(f"[ERROR] id: {row.id}：{e}")
        
        formula = structure.composition.reduced_formula
        print(f"\nid: {row.id}, formula: {formula}")
        
        structure_fixed = add_selective_dynamics_to_slab(structure.copy(), fixed_fraction=0.5)
        adsorbed_structure = add_adsorbate_model(structure_fixed, adsorbate, height=3.0)
        adsorbed_atoms = AseAtomsAdaptor.get_atoms(adsorbed_structure)
        
        out_db.write(
            adsorbed_atoms,
            key_value_pairs={
                "parent_id": row.id,
                "chem_formula": formula,
                "adsorbed": True
            },
            data={"adsorbed_structure": adsorbed_structure.as_dict()}
        )
        print(f" id: {row.id}  {output_db_file}")

if __name__ == "__main__":
    main()



正在处理结构 id: 1, formula: Hf6O11
结构 id: 1 的吸附模型已保存到 adsorbed.db

正在处理结构 id: 2, formula: Nb7O16
结构 id: 2 的吸附模型已保存到 adsorbed.db

正在处理结构 id: 3, formula: Sc32O45
结构 id: 3 的吸附模型已保存到 adsorbed.db

正在处理结构 id: 4, formula: Ta3O7
结构 id: 4 的吸附模型已保存到 adsorbed.db

正在处理结构 id: 5, formula: Ti3O5
结构 id: 5 的吸附模型已保存到 adsorbed.db

正在处理结构 id: 6, formula: V6O13
结构 id: 6 的吸附模型已保存到 adsorbed.db

正在处理结构 id: 7, formula: W4O7
结构 id: 7 的吸附模型已保存到 adsorbed.db

正在处理结构 id: 8, formula: Y32O39
结构 id: 8 的吸附模型已保存到 adsorbed.db

正在处理结构 id: 9, formula: ZrO2
结构 id: 9 的吸附模型已保存到 adsorbed.db
